1. Load and verify dataset

In [11]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('C:/Users/jsuri/Downloads/D/food/cleaned_bangalore_zomato_data.csv')

# Display column names
print(data.columns.tolist())  # Converts column names to a list for better readability


['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'multiple_deliveries', 'Time_taken_min', 'Time_Difference', 'Order_Hour', 'Distance', 'Weather_conditions_Cloudy', 'Weather_conditions_Fog', 'Weather_conditions_Sandstorms', 'Weather_conditions_Stormy', 'Weather_conditions_Sunny', 'Weather_conditions_Windy', 'Road_traffic_density_High', 'Road_traffic_density_Jam', 'Road_traffic_density_Low', 'Road_traffic_density_Medium', 'Vehicle_condition_0', 'Vehicle_condition_1', 'Vehicle_condition_2', 'Type_of_order_Buffet', 'Type_of_order_Drinks', 'Type_of_order_Meal', 'Type_of_order_Snack', 'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle', 'Type_of_vehicle_scooter']


2.  Extract Necessary Columns and Combine Unique Locations

In [14]:
# Extract necessary columns
restaurant_latitudes = data['Restaurant_latitude']
restaurant_longitudes = data['Restaurant_longitude']
delivery_latitudes = data['Delivery_location_latitude']
delivery_longitudes = data['Delivery_location_longitude']
time_taken = data['Time_taken_min']

In [16]:
# Combine restaurant and delivery locations into one list of unique locations
locations = np.unique(
    np.vstack((np.column_stack((restaurant_latitudes, restaurant_longitudes)), 
               np.column_stack((delivery_latitudes, delivery_longitudes)))),
    axis=0
)

In [18]:
# Number of unique locations
num_locations = locations.shape[0]


In [20]:
# Display the number of unique locations
print(f"Number of unique locations: {num_locations}")

Number of unique locations: 32


3. Preallocate Matrices and Initialize Constants

In [23]:
# Number of unique locations (assuming num_locations is already calculated)
distance_matrix = np.zeros((num_locations, num_locations))
time_matrix = np.zeros((num_locations, num_locations))

# Earth's radius in kilometers
earth_radius = 6371  # km

print("Distance and time matrices initialized with zeros.")

Distance and time matrices initialized with zeros.


4. Calculate Distance and Time Matrices with Adjustments

In [26]:
import numpy as np
import pandas as pd

# Earth's radius in kilometers
earth_radius = 6371  

# Convert degrees to radians
def deg2rad(degrees):
    return np.radians(degrees)

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = np.sin(dLat / 2) ** 2 + np.cos(deg2rad(lat1)) * np.cos(deg2rad(lat2)) * np.sin(dLon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return earth_radius * c

# Number of unique locations
num_locations = len(locations)

# Initialize distance and time matrices
distance_matrix = np.full((num_locations, num_locations), np.inf)
time_matrix = np.full((num_locations, num_locations), np.inf)

# Loop through each pair of unique locations
for i in range(num_locations):
    for j in range(num_locations):
        if i != j:
            # Extract coordinates
            lat1, lon1 = locations[i]
            lat2, lon2 = locations[j]

            # Calculate Haversine distance
            distance = haversine(lat1, lon1, lat2, lon2)

            # Initialize condition factor
            condition_factor = 1.0

            # Find the index of the delivery location corresponding to the current location
            delivery_index = np.where((delivery_latitudes == lat1) & (delivery_longitudes == lon1))[0]

            if len(delivery_index) > 0:
                delivery_index = delivery_index[0]  # Take the first match

                # Adjust condition factor based on weather conditions
                weather_conditions = {
                    "Weather_conditions_Cloudy": 1.1,
                    "Weather_conditions_Fog": 1.3,
                    "Weather_conditions_Sandstorms": 1.4,
                    "Weather_conditions_Stormy": 1.5,
                    "Weather_conditions_Sunny": 0.9,
                    "Weather_conditions_Windy": 1.2,
                }
                
                for condition, factor in weather_conditions.items():
                    if data.loc[delivery_index, condition] == 1:
                        condition_factor *= factor
                
                # Adjust condition factor based on traffic conditions
                traffic_conditions = {
                    "Road_traffic_density_Jam": 2.0,
                    "Road_traffic_density_High": 1.3,
                    "Road_traffic_density_Low": 0.8,
                    "Road_traffic_density_Medium": 1.1,
                }

                for condition, factor in traffic_conditions.items():
                    if data.loc[delivery_index, condition] == 1:
                        condition_factor *= factor

                # Calculate adjusted distance and time
                adjusted_distance = distance * condition_factor
                distance_matrix[i, j] = adjusted_distance

                adjusted_time = data.loc[delivery_index, "Time_taken_min"] * condition_factor
                time_matrix[i, j] = adjusted_time


5. Save Matrices and Visualize Paths

In [35]:
import folium
import pandas as pd
import numpy as np

# Load your dataset
data = pd.read_csv("cleaned_bangalore_zomato_data.csv")

# Extract the necessary columns
restaurant_latitudes = data["Restaurant_latitude"]
restaurant_longitudes = data["Restaurant_longitude"]
delivery_latitudes = data["Delivery_location_latitude"]
delivery_longitudes = data["Delivery_location_longitude"]

# Combine unique locations
locations = np.unique(np.vstack((np.column_stack((restaurant_latitudes, restaurant_longitudes)), 
                                 np.column_stack((delivery_latitudes, delivery_longitudes)))), axis=0)

# Create a folium map centered around Bangalore
m = folium.Map(location=[np.mean(restaurant_latitudes), np.mean(restaurant_longitudes)], zoom_start=12)

# Add restaurant markers
for lat, lon in zip(restaurant_latitudes, restaurant_longitudes):
    folium.Marker([lat, lon], icon=folium.Icon(color="red"), popup="Restaurant").add_to(m)

# Add delivery location markers
for lat, lon in zip(delivery_latitudes, delivery_longitudes):
    folium.Marker([lat, lon], icon=folium.Icon(color="green"), popup="Delivery Location").add_to(m)

# Add lines connecting locations (delivery paths)
for i in range(len(locations)):
    for j in range(len(locations)):
        if i != j:  # Avoid self-connections
            folium.PolyLine([locations[i], locations[j]], color="blue", weight=1.5).add_to(m)

# Save and display the map
m.save("delivery_paths_map.html")
m  # This will render the interactive map in Jupyter Notebook


In [33]:
!pip install folium


   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/110.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/110.9 kB ? eta -:--:--
   ---------- ---------------------------- 30.7/110.9 kB 187.9 kB/s eta 0:00:01
   ---------- ---------------------------- 30.7/110.9 kB 187.9 kB/s eta 0:00:01
   ---------- ---------------------------- 30.7/110.9 kB 187.9 kB/s eta 0:00:01
   ---------- ---------------------------- 30.7/110.9 kB 187.9 kB/s eta 0:00:01
   ---------------------------- ---------- 81.9/110.9 kB 218.5 kB/s eta 0:00:01
   -------------------------------- ------ 92.2/110.9 kB 218.5 kB/s eta 0:00:01
   -------------------------------------- 110.9/110.9 kB 238.5 kB/s eta 0:00:00
